### Reinforcement Learning: Theory and Algorithms Chapter 14 CPI

* 第14章における各定理、補題、系などをまとめます。証明は初見でもこれを参考にすれば導出できるように丁寧に記します。
* 必要に応じて実装を行います。

保守的方策反復(CPI)と信頼領域制約方策最適化(TRPO)、近位方策最適化(PPO)はいずれも、方策の更新について、現在の方策から更新後の方策があまり離れないように強制する手法であり、小さな増分更新を行うものです。特に、PPOはロボットの制御方策を獲得するための研究で頻繁に用いられており、実用的な手法の一つです。では、なぜ方策の増分更新を少なくすることが、最適方策を獲得する上で重要なのでしょうか。それをこのページでは議論していきます。

### 準備
*  $\{\mathcal{S}, \mathcal{A}, P, r, \gamma, \rho\}$ ：割引無限MDP
* $\rho$：初期状態の分布
* $\mu$：

### 保守的方策反復(CPI)

最初に、CPIのアルゴリズムを見てみましょう。


ここで、方策の更新は、ステップサイズ$\alpha$によって制御されています。$1-\alpha$分だけ、現在の方策を減らし、$\alpha$分だけ方策を更新します。

#### 保守的な方策の更新
$\alpha$の重要性を知るために、パフォーマンス補題を以下のように示します。

この差分が大きいほど、更新量が多いため、$\mathbb{E}_{s \sim d_\mu^{\pi^{t+1}}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)$ を最大化するような方策$\pi^{\prime} \in \Pi$を使えば、方策が改善されることを保証できます。しかし、エピソード $t$ では、$d_\mu^{\pi^{t+1}}$は未来の情報のため、取得できません。$d_\mu^{\pi^t}$は取得できます。$d_\mu^{\pi^t}$をそこまで大きく変化させないことを考えます。つまり、$d_\mu^{\pi^{t+1}}$と$d_\mu^{\pi^t}$の差をそこまで大きくしないわけです。それにより、$\mathbb{E}_{s \sim d_\mu^{\pi^{t+1}}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)$ を、$\mathbb{E}_{s \sim d \pi^t} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)$と見なせるので、これを最大化する方策を探索することができます。$d_\mu^{\pi^{t+1}}$と$d_\mu^{\pi^t}$の差の大きさは、方策の差の大きさと比例しており、方策を微小に更新することで、$d_\mu^{\pi^{t+1}}$と$d_\mu^{\pi^t}$の差を小さくすることできます。

補題14.1は、方策が総変動距離の意味で近い場合、その占有測度も近いことを示しており、それを裏付けています。

****
**補題 14.1** (類似した方策は類似した状態訪問を意味する)任意の $t$ について、以下が成り立つ。
$$
\left\|\pi^{t+1}(\cdot \mid s)-\pi^t(\cdot \mid s)\right\|_1 \leq 2 \alpha, \forall s
$$

さらに、以下が成り立つ。
$$
\left\|d_\mu^{\pi^{t+1}}-d_\mu^{\pi^t}\right\|_1 \leq \frac{2 \alpha \gamma}{1-\gamma}
$$
****
**補題14.1の証明**
まず、補題の最初の主張として、方策更新による方策自体の変化の評価を行います。これは、方策更新の定義から直接導かれます。
更新ルールに従い、 $\pi^{t+1}$ は$\pi^t$ と、新しく選ばれた貪欲方策 $\pi^{\prime}$ の重み付き平均として定義されます。

$$
\pi^{t+1} = (1 - \alpha) \pi^t + \alpha \pi^{\prime}
$$

ここで、任意の状態 $s$ において、方策 $\pi^{t+1}$ と $\pi^t$ の行動確率の差の $\ell_1$ ノルム、すなわち $\left\|\pi^{t+1}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1$ を計算します。

$$
\left\|\pi^{t+1}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1 = \left\|(1 - \alpha) \pi^t(\cdot \mid s) + \alpha \pi^{\prime}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1
$$

$$
= \left\| \alpha \pi^{\prime}(\cdot \mid s) - \alpha \pi^t(\cdot \mid s) \right\|_1
$$

$$
= \alpha \left\|\pi^{\prime}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1
$$

確率分布の $\ell_1$ ノルムは、全確率が1であることから、最大値が2となります。つまり、$\left\|\pi^{\prime}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1 \leq 2$ が成り立ちます。したがって、

$$
\left\|\pi^{t+1}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1 \leq 2\alpha
$$

が成り立ちます。これは、方策の更新による変化が、 $\alpha$ に比例して小さいことを示しています。

$\mathbb{P}_h^\pi(s)$ を、初期状態分布 $\mu$ から開始し、タイムステップ $h$ で方策 $\pi$ に従った場合の状態分布と定義します。つまり、$\mathbb{P}_h^\pi(s)$ は、時間ステップ $h$ において、状態 $s$ に到達する確率を表します。

方策が $\pi^t$ から $\pi^{t+1}$ に変化したときの状態分布の変化 $\mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t}$ を求めます。状態分布は再帰的に定義されるため、以下のように展開できます。

まず、状態 $s^{\prime}$ における差分を考えます。

$$
\mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) = \sum_{s, a} \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \pi^{t+1}(a \mid s) P(s^{\prime} \mid s, a)
$$

$$
\mathbb{P}_h^{\pi^t}(s^{\prime}) = \sum_{s, a} \mathbb{P}_{h-1}^{\pi^t}(s) \pi^t(a \mid s) P(s^{\prime} \mid s, a)
$$

したがって、差分は、

$$
\mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime})
= \sum_{s, a} \left( \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \pi^{t+1}(a \mid s) - \mathbb{P}_{h-1}^{\pi^t}(s) \pi^t(a \mid s) \right) P(s^{\prime} \mid s, a)
$$

この式を展開するために、右辺に項を足し引きします。

$$
\begin{aligned}
\mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime})
&= \sum_{s, a} \left( \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \pi^{t+1}(a \mid s) - \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \pi^t(a \mid s) \right) P(s^{\prime} \mid s, a) \\
& \quad + \sum_{s, a} \left( \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \pi^t(a \mid s) - \mathbb{P}_{h-1}^{\pi^t}(s) \pi^t(a \mid s) \right) P(s^{\prime} \mid s, a) \\
&= \sum_{s, a} \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \left( \pi^{t+1}(a \mid s) - \pi^t(a \mid s) \right) P(s^{\prime} \mid s, a) \\
& \quad + \sum_{s, a} \left( \mathbb{P}_{h-1}^{\pi^{t+1}}(s) - \mathbb{P}_{h-1}^{\pi^t}(s) \right) \pi^t(a \mid s) P(s^{\prime} \mid s, a)
\end{aligned}
$$

この式の両辺の絶対値をとることで、$\ell_1$ ノルムの評価に進みます。

$$
\begin{aligned}
\sum_{s^{\prime}} \left| \mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime}) \right|
& \leq \sum_{s^{\prime}} \left| \sum_{s, a} \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \left( \pi^{t+1}(a \mid s) - \pi^t(a \mid s) \right) P(s^{\prime} \mid s, a) \right| \\
& \quad + \sum_{s^{\prime}} \left| \sum_{s, a} \left( \mathbb{P}_{h-1}^{\pi^{t+1}}(s) - \mathbb{P}_{h-1}^{\pi^t}(s) \right) \pi^t(a \mid s) P(s^{\prime} \mid s, a) \right|
\end{aligned}
$$

三角不等式を用いて、

$$
\begin{aligned}
\sum_{s^{\prime}} \left| \mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime}) \right|
& \leq \sum_s \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \sum_a \left| \pi^{t+1}(a \mid s) - \pi^t(a \mid s) \right| \sum_{s^{\prime}} P(s^{\prime} \mid s, a) \\
& \quad + \sum_s \left| \mathbb{P}_{h-1}^{\pi^{t+1}}(s) - \mathbb{P}_{h-1}^{\pi^t}(s) \right| \sum_a \pi^t(a \mid s) \sum_{s^{\prime}} P(s^{\prime} \mid s, a).
\end{aligned}
$$

ここで、$\sum_{s^{\prime}} P(s^{\prime} \mid s, a) = 1$ および $\sum_a \pi^t(a \mid s) = 1$ であること、さらに上で示した $\left\|\pi^{t+1}(\cdot \mid s) - \pi^t(\cdot \mid s)\right\|_1 \leq 2\alpha$ を代入すると、

$$
\sum_{s^{\prime}} \left| \mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime}) \right| \leq \sum_s \mathbb{P}_{h-1}^{\pi^{t+1}}(s) \cdot 2\alpha + \sum_s \left| \mathbb{P}_{h-1}^{\pi^{t+1}}(s) - \mathbb{P}_{h-1}^{\pi^t}(s) \right|.
$$

さらに、$\sum_s \mathbb{P}_{h-1}^{\pi^{t+1}}(s) = 1$ であることから、

$$
\sum_{s^{\prime}} \left| \mathbb{P}_h^{\pi^{t+1}}(s^{\prime}) - \mathbb{P}_h^{\pi^t}(s^{\prime}) \right| \leq 2\alpha + \sum_s \left| \mathbb{P}_{h-1}^{\pi^{t+1}}(s) - \mathbb{P}_{h-1}^{\pi^t}(s) \right|.
$$

これは、$\ell_1$ ノルムの定義に基づき、

$$
\left\| \mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t} \right\|_1 \leq 2\alpha + \left\| \mathbb{P}_{h-1}^{\pi^{t+1}} - \mathbb{P}_{h-1}^{\pi^t} \right\|_1.
$$

と書き換えられます。この不等式を繰り返し適用することで、

$$
\left\| \mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t} \right\|_1 \leq 2\alpha h
$$
ではなく
$$
\left\| \mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t} \right\|_1 \leq 2h\alpha
$$

が導かれます。 $d_\mu^\pi$ は、タイムステップごとの状態分布 $\mathbb{P}_h^\pi$ の重み付き和で表されます。具体的には、 $\gamma$ を用いて以下のように定義されます。

$$
d_\mu^{\pi}(s) = (1 - \gamma) \sum_{h=0}^{\infty} \gamma^h \mathbb{P}_h^\pi(s)
$$

したがって、方策が $\pi^t$ から $\pi^{t+1}$ に変化したとき、状態分布の変化は次のようになります。

$$
d_\mu^{\pi^{t+1}} - d_\mu^{\pi^t} = (1 - \gamma) \sum_{h=0}^{\infty} \gamma^h (\mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t}).
$$

この両辺の $\ell_1$ ノルムをとると、

$$
\left\| d_\mu^{\pi^{t+1}} - d_\mu^{\pi^t} \right\|_1 = \left\| (1 - \gamma) \sum_{h=0}^{\infty} \gamma^h (\mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t}) \right\|_1
$$

三角不等式を用いて、

$$
\left\| d_\mu^{\pi^{t+1}} - d_\mu^{\pi^t} \right\|_1 \leq (1 - \gamma) \sum_{h=0}^{\infty} \gamma^h \left\| \mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t} \right\|_1.
$$

上で導出した不等式 $\left\| \mathbb{P}_h^{\pi^{t+1}} - \mathbb{P}_h^{\pi^t} \right\|_1 \leq 2 h \alpha$ を代入すると、

$$
\left\| d_\mu^{\pi^{t+1}} - d_\mu^{\pi^t} \right\|_1 \leq (1 - \gamma) \sum_{h=0}^{\infty} \gamma^h (2 h \alpha)
$$

$$
= 2 \alpha (1 - \gamma) \sum_{h=0}^{\infty} h \gamma^h.
$$

ここで、以下の公式を利用します。

$$
\sum_{h=0}^{\infty} h \gamma^h = \frac{\gamma}{(1-\gamma)^2}.
$$

これを代入すると、

$$
\left\| d_\mu^{\pi^{t+1}} - d_\mu^{\pi^t} \right\|_1 \leq 2 \alpha (1 - \gamma) \frac{\gamma}{(1-\gamma)^2}
$$

$$
= \frac{2 \alpha \gamma}{1 - \gamma}.
$$

これにより、状態訪問分布 $d_\mu^{\pi^t}$ から $d_\mu^{\pi^{t+1}}$ への変化が、$\alpha$ のスケールで有界であることが示されました。具体的には、変化の大きさは、 $\alpha$ と $\gamma$ に依存し、$\frac{2 \alpha \gamma}{1 - \gamma}$ によって制限されます。

****


#### CPIの単調改善
つぎに、CPIが単調にいい感じに改善されることを確認します。$\mathbb{A}=\mathbb{E}_{s \sim d_\mu^{\pi^t}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)$ とします。$\mathbb{A}>0$である限り、$\pi^{t+1}$ は $\pi^t$ よりも厳密に良いことを定理14.2で確認しましょう。

****
**定理 14.2 (CPI における単調な改善)**

任意のエピソード $t$ について考える。$\mathbb{A}=\mathbb{E}_{s \sim d_\mu^{\pi^t}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)$ とする。以下が成り立つ。
$$
V_\mu^{\pi^{t+1}}-V_\mu^{\pi^t} \geq \frac{\alpha}{1-\gamma}\left(\mathbb{A}-\frac{2 \alpha \gamma}{(1-\gamma)^2}\right)
$$
$\alpha=\frac{\mathrm{A}(1-\gamma)^2}{4 \gamma}$ と設定すると、以下が得られる。
$$
V_\mu^{\pi^{t+1}}-V_\mu^{\pi^t} \geq \frac{\mathbb{A}^2(1-\gamma)}{8 \gamma}
$$
****
**定理14.2の証明**
パフォーマンス補題を適用すると、方策 $\pi^t$ から $\pi^{t+1}$ への性能の変化は次のように表されます。

$$
V_\mu^{\pi^{t+1}} - V_\mu^{\pi^t} = \frac{1}{1-\gamma} \mathbb{E}_{s \sim d_\mu^{\pi^{t+1}}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right)
$$

ここで、$\pi^{t+1}$ は $\pi^t$ と貪欲方策 $\pi^{\prime}$ の混合方策であり、$\pi^{\prime}$ は $\pi^t$ に対する近似貪欲方策です。

補題 14.1 によると、 $d_\mu^{\pi^{t+1}}$ の変化を考慮すると、次の式が得られます。

$$
\begin{aligned}
(1-\gamma) \left(V_\mu^{\pi^{t+1}} - V_\mu^{\pi^t} \right) &= \mathbb{E}_{s \sim d_\mu^{\pi^t}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right) \\
&\quad + \mathbb{E}_{s \sim d_\mu^{\pi^{t+1}}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right) - \mathbb{E}_{s \sim d_\mu^{\pi^t}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right).
\end{aligned}
$$

ここで、2 番目の期待値項は、新しい状態分布 $d_\mu^{\pi^{t+1}}$ における期待値であり、これは $d_\mu^{\pi^t}$ での期待値に分布のずれの影響を加味して近似できます。

この影響を考慮すると、次の不等式が成り立ちます。

$$
\mathbb{E}_{s \sim d_\mu^{\pi^{t+1}}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right)
\geq \mathbb{E}_{s \sim d_\mu^{\pi^t}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right) - \sup_{s, a, \pi} \left| A^\pi(s, a) \right| \left\| d_\mu^{\pi^t} - d_\mu^{\pi^{t+1}} \right\|_1.
$$

この式は、分布 $P_1$ と $P_2$、および任意の関数 $f$ に対して

$$
\left| \mathbb{E}_{x \sim P_1} f(x) - \mathbb{E}_{x \sim P_2} f(x) \right| \leq \sup_{x} \left| f(x) \right| \cdot \| P_1 - P_2 \|_1
$$

という関係が成り立つことを利用したものです。補題 14.1 の結果を適用すると、状態分布の変化は以下のように抑えられます。

$$
\| d_\mu^{\pi^t} - d_\mu^{\pi^{t+1}} \|_1 \leq \frac{2 \alpha \gamma}{1 - \gamma}.
$$

また、任意の方策 $\pi$ に対して、アドバンテージ関数 $A^\pi(s, a)$ は次の上限を持ちます。

$$
\sup_{s, a, \pi} \left| A^\pi(s, a) \right| \leq \frac{1}{1-\gamma}.
$$

したがって、不等式は次のように変形されます。

$$
\begin{aligned}
(1-\gamma)(V_\mu^{\pi^{t+1}} - V_\mu^{\pi^t})
&\geq \mathbb{E}_{s \sim d_\mu^{\pi^t}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right) - \frac{\alpha}{1-\gamma} \left\| d_\mu^{\pi^t} - d_\mu^{\pi^{t+1}} \right\|_1 \\
&\geq \mathbb{E}_{s \sim d_\mu^{\pi^t}} \alpha A^{\pi^t}\left(s, \pi^{\prime}(s)\right) - \frac{2 \alpha^2 \gamma}{(1-\gamma)^2}.
\end{aligned}
$$

ここで、最後の不等式は補題 14.1 を適用して得られます。

以上の結果を整理すると、次の不等式が成り立ちます。

$$
V_\mu^{\pi^{t+1}} - V_\mu^{\pi^t} \geq \frac{\alpha}{1-\gamma} \left( \mathbb{E}_{s \sim d_\mu^{\pi^t}} A^{\pi^t}\left(s, \pi^{\prime}(s)\right) - \frac{2 \alpha \gamma}{(1-\gamma)^2} \right).
$$
****

#### CPIの最適性

単調に改善することを示したので、最適性も示しておきたいところです。最初に局所的な最適性を示します。

*****

**定理 14.3 (CPI の局所的な最適性)**
CPIアルゴリズムは、最大で $8 \gamma / \epsilon^2$ ステップで終了し、$\max _{\pi \in \Pi} \mathbb{E}_{s \sim d_\mu^{\pi^t}} A^{\pi^t}(s, \pi(s)) \leq 2 \varepsilon$ を満たす方策 $\pi^t$ を出力する。

****
定理 14.3 の証明は難しくないので、RL理論本を参照しましょう。$\mu$が $d^{\pi^{\star}}$ をカバーする場合、つまり、最適方策が訪問する状態の頻度が、$\mu$でいい感じにカバーされるとき、大域的な最適性を示すことができます。

****
**定理 14.4 (CPI のグローバルな最適性)**
終了時に、以下を満たす方策 $\pi$ が得られる。
$$
V^{\star}-V^\pi \leq \frac{2 \varepsilon+\epsilon_{\Pi}}{(1-\gamma)^2}\left\|\frac{d^{\pi^{\star}}}{\mu}\right\|_{\infty},
$$
ここで、$\epsilon_{\Pi}:=\mathbb{E}_{s \sim d_\mu^\pi}\left[\max _{a \in \mathcal{A}} A^\pi(s, a)\right]-\max _{\pi \in \Pi} \mathbb{E}_{s \sim d_\mu^\pi}\left[A^\pi(s, \pi(s))\right]$ である。
****
**定理14.4の証明**
パフォーマンス補題により、
$$
\begin{aligned}
V^{\star}-V^\pi & =\frac{1}{1-\gamma} \mathbb{E}_{s \sim d^\pi} A^\pi\left(s, \pi^{\star}(s)\right) \\
& \leq \frac{1}{1-\gamma} \mathbb{E}_{s \sim d^{\pi^{\star}}} \max _{a \in \mathcal{A}} A^\pi(s, a) \\
& \leq \frac{1}{1-\gamma}\left\|\frac{d^{\pi^*}}{d_\mu^\pi}\right\|_{\infty} \mathbb{E}_{s \sim d_\mu^\pi} \max _{a \in \mathcal{A}} A^\pi(s, a) \\
& \leq \frac{1}{(1-\gamma)^2}\left\|\frac{d^{\pi^*}}{\mu}\right\|_{\infty} \mathbb{E}_{s \sim d_\mu^\pi} \max _{a \in \mathcal{A}} A^\pi(s, a) \\
& =\leq \frac{1}{(1-\gamma)^2}\left\|\frac{d^{\pi^*}}{\mu}\right\|_{\infty}\left[\max _{\hat{\pi} \in \Pi} \mathbb{E}_{s \sim d_\mu^\pi} A^\pi(s, \hat{\pi}(s))-\max _{\hat{\pi} \in \Pi} \mathbb{E}_{s \sim d_\mu^\pi} A^\pi(s, \hat{\pi}(s))+\mathbb{E}_{s \sim d_\mu^\pi} \max _{a \in \mathcal{A}} A^\pi(s, a)\right] \\
& \leq \frac{1}{(1-\gamma)^2}\left\|\frac{d^{\pi^*}}{\mu}\right\|_{\infty}\left(2 \varepsilon+\epsilon_{\Pi}\right),
\end{aligned}
$$
2 番目の不等式は $\max _a A^\pi(s, a) \geq 0$ 、3 番目の不等式は任意の $s$ と $\pi$ に対して $d_\mu^\pi(s) \geq(1-\gamma) \mu(s)$ という事実を使用し、最後の不等式は $\epsilon_{\Pi}$ の定義と定理 14.3 を使用しました。

#### CPIと方策勾配法の比較

| **比較項目**              | **CPI**                                             | **方策勾配法**                            |
| --------------------- | --------------------------------------------------- | ------------------------------------ |
| **局所的な最適性保証**         | 常に局所的最適性を満たす                                        | 方策パラメータが滑らかで価値関数が微分可能な場合に成り立つ        |
| **near-optimal方策の生成** | 最適方策との分布の差分が小さい場合、ほぼ最適な方策を得られる                      | 同様                                   |
| **改善のメカニズム**          | アドバンテージ関数の最大化                                       | 勾配上昇による局所的な改善                        |
| **方策の制約**             | **なし** (任意の方策クラスに適用可能)                              | **あり** (方策がパラメータの微分可能な関数であることが必要)    |
| **非微分可能な方策の扱い**       | 可能 (教師あり学習への還元を利用)                                  | **難しい** (回帰木などの非微分可能な方策には適用困難)       |
| **更新時の局所性**           | **方策の混合($1-\alpha$と$\alpha$の割合で混ぜる)** により状態分布の変化を制御 | **パラメータ空間での局所的な更新** により状態分布の変化を小さくする |
| **分散の問題**             | 方策の保守的な混合により分散の増大を抑える                               | 分散が大きくなりやすく、制御変量や信頼領域手法が必要           |
| **適用範囲**              | 汎用的で広範な方策クラスに適用可能                                   | 微分可能なパラメータ化が前提のモデルに適用可能              |
| **信頼領域制約**            | なし (方策の混合で状態分布の変化を抑制)                               | 信頼領域手法 (TRPO など) を利用して状態分布の変化を制御     |

